In [ ]:
%tensorflow_version 1.x

# Clone repo

In [ ]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

In [ ]:
!git checkout equal_realize

In [ ]:
!git pull

# Amount GDrive

In [ ]:
from driver_amount import addh

# Install python package

In [ ]:
!pip install keras_bert

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")

In [ ]:
os.chdir("bachelor_design")

# Get input

In [ ]:
from cut_and_tag import load_stopwords, cut_and_remove_stopwords
import config

In [ ]:
input_strs = None
# TODO load input_strs

stopwords = load_stopwords(addh + config.STOPWORDS_PATH)
cut_seqs = []
char_seqs = []
for input_str in input_strs:
    cut_seq = cut_and_remove_stopwords(input_str, stopwords)
    cut_seqs.append(cut_seq)
    
    char_seq = []
    for w in cut_seq:
        for c in w:
            char_seq.append(c)
    char_seqs.append(char_seq)

# Load params

In [ ]:
import json

In [ ]:
tag_vocab = None
with open(addh + config.TAG_VOCAB_PATH, "r") as fd:
    tag_vocab = json.load(fd)

In [ ]:
equal_params = None
with open(addh + config.EQUAL_PARAM_PATH, "r") as fd:
    equal_params = json.load(fd)

# Identify equal

In [ ]:
from char_emb import CharEmbedder
from SIF import SIF
from dist_cal import DistCal

In [ ]:
emb_method = "sum_last_four"
dist_method = "cos"
dist_theta = equal_params[dist_method + "_theta"] # TODO edit equal params

In [ ]:
char_embedder = CharEmbedder()
sif = SIF(params["sif_alpha"])

## Embedding

In [ ]:
char_emb_seqs = char_embedder.embed(char_seqs, emb_method)
sen_vecs = sif.compose(cut_seqs, char_seqs, char_emb_seqs)

## Cal similarity and remove

In [ ]:
dist_cal = DistCal(sen_vecs)

In [ ]:
removed_index = []

In [ ]:
for i in range(len(sen_vecs)-1):
    if i in removed_index:
        continue
    for j in range(i+1, len(sen_vecs)):
        if j in removed_index:
            continue
        
        dist = dist_cal.cal(sen_vecs[i], sen_vecs[j], dist_method)
        if dist < dist_theta:
            removed_index.push(j)

In [ ]:
new_char_seqs = []
for i in range(len(char_seqs)):
    if i in removed_index:
        continue
    new_char_seqs.push(char_seqs[i])
char_seqs = new_char_seqs

# Identify function points

## Load model

In [ ]:
import keras
import keras_bert
from keras_contrib.layers import CRF

In [ ]:
bert_model, bert_model_config = keras_bert.build_model_from_config(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True
)

In [ ]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [ ]:
model.load_weights(addh + config.MODEL_PATH)

## Preprocess

In [ ]:
import preprocess

In [ ]:
vocab = keras_bert.load_vocabulary(addh + config.BERT_VOCAB_PATH)

token_id_seqs = preprocess.preprocess_char(
    char_seqs,
    vocab,
    config.SEQ_LEN,
    True
)
segment_seqs = preprocess.create_segment(
    len(token_id_seqs),
    len(token_id_seqs[0])
)

## Predict

In [ ]:
batch_size = 16
completed = False
import tensorflow as tf

In [ ]:
output_one_hot_seqs = None
while not completed:
    try:
        output_one_hot_seqs = model.predict(
            [token_id_seqs, segment_seqs],
            batch_size = batch_size,
            verbose=1
        )
        completed = True
    except tf.errors.ResourceExhaustedError as e:
        batch_size = int(batch_size/2)
        print("Batch Size too large, turn to " + str(batch_size))
        if batch_size < 1:
            raise e

## Postprocess

In [ ]:
from epoch_checkpoint import judgeWhichTag

In [ ]:
# Turn to tag id
output_seqs = []
for one_hot_seq in output_one_hot_seqs:
    one_hot_seq = one_hot_seq[1:-1] # Remove [CLS] and [SEP]
    seq = [[np.argmax(x)] for x in one_hot_seq]
    output_seqs.append(seq)

In [ ]:
rev_tag_vocab = {}
for key, value in tag_vocab.items():
    rev_tag_vocab[value] = key

In [ ]:
# Judge each output seq's tag
output_tags = [judgeWhichTag(seq, rev_tag_vocab) for seq in output_seqs]

In [ ]:
# Count each tag category's number
tag_num = {}
for tag in output_tags:
    num = tag_num.get(tag, 0)
    tag_num[tag] = num + 1

In [ ]:
fp_counts = tag_num

# Calculate cost

In [ ]:
from funcomo import FUNCOMO

## Set model params

In [ ]:
cost_model_params = {
    "fp_p" : FUNCOMO.fp_p,
    "fp_q" : FUNCOMO.fp_q,
    "VAF" : [2, 2],

    "lang" : "C++",
    "lang_factor" : FUNCOMO.lang_factor[lang],

    "develop_mode" : "semi_detached",
    "develop_mode_factor" : FUNCOMO.develop_mode_factor[develop_mode],
    "em" : "early_design",
    "em_factor" : FUNCOMO.EM_factor[em],
    "KLOC_p" : FUNCOMO.KLOC.p,
    "KLOC_q" : FUNCOMO.KLOC.q,
    "w" : [2, 2, 2, 2, 2],
}

## Use cost model

In [ ]:
cmp = cost_model_params

scale = FUNCOMO.FP2Scale(
    fp_counts,
    cmp["fp_p"],
    cmp["fp_q"],
    cmp["VAF"],
)

KLOC = FUNCOMO.scale2KLOC(
    cmp["scale"],
    cmp["lang_factor"],
)

PM = FUNCOMO.KLOC2PM(
    cmp["KLOC"],
    cmp["develop_mode_factor"],
    cmp["em_factor"],
    cmp["KLOC_p"],
    cmp["KLOC_q"],
    cmp["w"],
)

## Output result

In [ ]:
from pprint import PrettyPrinter

In [ ]:
pp = PrettyPrinter(indent = 4)

In [ ]:
print("Using params: ")
pp.pprint(cost_model_params)

In [ ]:
print("Result: ")
pp.pprint({
    "KLOC": KLOC,
    "PM": PM,
})